# Meta-Sampling: Estratégia de seleção de instâncias baseada em meta-aprendizado

## Elaboração de meta-base 

In [1]:
DATA_DIR = '../../metabase/'

In [2]:
%%time
import os
import pandas as pd

csv_files =  [file for file in os.listdir(DATA_DIR) if file.endswith('csv')]
df_list = [pd.read_csv(os.path.join(DATA_DIR, file), index_col=False) for file in csv_files]

CPU times: user 13.9 s, sys: 7.65 s, total: 21.5 s
Wall time: 13.9 s


Juntando dataframes:

In [11]:
df = pd.concat(df_list, keys=csv_files)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.index.names=['dataset', 'index']

Adicionando a informação referente à query:

In [13]:
def add_query_index(x):
    x['query_index']=range(len(x))
    return x
    
df = df.groupby(['dataset','estimator', 'query-strategy'], group_keys=False).apply(add_query_index)
df

estimator             query-strategy  \
dataset                      index                                          
1016_vowel.csv               0       SVCLinear       uncertainty_sampling   
                             1       SVCLinear       uncertainty_sampling   
                             2       SVCLinear       uncertainty_sampling   
                             3       SVCLinear       uncertainty_sampling   
                             4       SVCLinear       uncertainty_sampling   
...                                        ...                        ...   
44153_vehicle_reproduced.csv 5595   GaussianNB  max_disagreement_sampling   
                             5596   GaussianNB  max_disagreement_sampling   
                             5597   GaussianNB  max_disagreement_sampling   
                             5598   GaussianNB  max_disagreement_sampling   
                             5599   GaussianNB  max_disagreement_sampling   

                                    accuracy  f1-micro  f1-macro  f1-weighted  \
dataset                      index                                              
1016_vowel.csv               0      0.935484  0.935484  0.760386     0.943743   
                             1      0.939516  0.939516  0.816704     0.940121   
                             2      0.931452  0.931452  0.783562     0.933616   
                             3      0.955645  0.955645  0.839369     0.960605   
                             4      0.963710  0.963710  0.890022     0.964073   
...                                      ...       ...       ...          ...   
44153_vehicle_reproduced.csv 5595   0.481132  0.481132  0.437514     0.526291   
                             5596   0.495283  0.495283  0.451377     0.540403   
                             5597   0.495283  0.495283  0.457154     0.534831   
                             5598   0.485849  0.485849  0.443711     0.529263   
                             5599   0.476415  0.476415  0.427293     0.526856   

                                    attr_conc.mean  attr_conc.sd  \
dataset                      index                                 
1016_vowel.csv               0            0.015592      0.018493   
                             1            0.027417      0.032775   
                             2            0.051948      0.125657   
                             3            0.023783      0.028042   
                             4            0.024757      0.032957   
...                                            ...           ...   
44153_vehicle_reproduced.csv 5595         0.182956      0.205057   
                             5596         0.158299      0.160699   
                             5597         0.193431      0.154040   
                             5598         0.167582      0.145910   
                             5599         0.144528      0.124435   

                                    attr_ent.mean  attr_ent.sd  ...       vdb  \
dataset                      index                              ...             
1016_vowel.csv               0           1.444044     1.359405  ...  1.954437   
                             1           1.444003     1.359415  ...  2.014466   
                             2           1.381090     1.276496  ...  2.038143   
                             3           1.381063     1.276569  ...  2.037769   
                             4           1.381054     1.276570  ...  2.024449   
...                                           ...          ...  ...       ...   
44153_vehicle_reproduced.csv 5595        2.293674     0.061258  ...  3.655533   
                             5596        2.296339     0.050446  ...  3.539950   
                             5597        2.297229     0.047935  ...  3.479908   
                             5598        2.296033     0.053981  ...  3.659758   
                             5599        2.297103     0.051473  ...  3.089899   

                                         v

Selecionando o classificador SVM (`SVC`) para teste

In [17]:
df_svc = df.loc[df.estimator=='SVC']

In [16]:
blau_df = df_svc.loc[df_svc.index.levels[0][0]]
blau_df.groupby('query_index').apply(lambda x: x)
# blau_df.groupby('q_number').apply(lambda x: x.loc[x['accuracy'].idxmax()])

estimator              query-strategy  accuracy  f1-micro  \
query_index index                                                             
0           700         SVC        uncertainty_sampling  0.907258  0.907258   
            800         SVC  uncertainty_batch_sampling  0.907258  0.907258   
            900         SVC             margin_sampling  0.923387  0.923387   
            1000        SVC            entropy_sampling  0.907258  0.907258   
            1100        SVC       vote_entropy_sampling  0.923387  0.923387   
...                     ...                         ...       ...       ...   
99          999         SVC             margin_sampling  0.991935  0.991935   
            1099        SVC            entropy_sampling  0.991935  0.991935   
            1199        SVC       vote_entropy_sampling  0.991935  0.991935   
            1299        SVC  consensus_entropy_sampling  1.000000  1.000000   
            1399        SVC   max_disagreement_sampling  0.979839  0.979839   

                   f1-macro  f1-weighted  attr_conc.mean  attr_conc.sd  \
query_index index                                                        
0           700    0.475687     0.951374        0.017823      0.024257   
            800    0.475687     0.951374        0.019425      0.019067   
            900    0.805731     0.917901        0.015242      0.019191   
            1000   0.475687     0.951374        0.024928      0.028208   
            1100   0.811693     0.916954        0.028235      0.034213   
...                     ...          ...             ...           ...   
99          999    0.975060     0.992101        0.034417      0.045034   
            1099   0.975060     0.992101        0.030359      0.047647   
            1199   0.976039     0.991935        0.048373      0.125203   
            1299   1.000000     1.000000        0.034597      0.043608   
            1399   0.941241     0.979647        0.038609      0.034962   

                   attr_ent.mean  attr_ent.sd  ...       vdb       vdu  \
query_index index                              ...                       
0           700         1.444044     1.359405  ...  1.954437  0.000003   
            800         1.444044     1.359405  ...  1.954437  0.000003   
            900         1.444044     1.359405  ...  1.954437  0.000003   
            1000        1.444044     1.359405  ...  1.954437  0.000003   
            1100        1.444044     1.359405  ...  1.954437  0.000003   
...                          ...          ...  ...       ...       ...   
99          999         1.221135     1.081386  ...  1.046266  0.000361   
            1099        1.221015     1.081598  ...  0.994977  0.000318   
            1199        1.226730     1.074924  ...  1.917423  0.000025   
            1299        1.221597     1.080939  ...  0.982915  0.000572   
            1399        1.225866     1.076750  ...  1.950639  0.000020   

                   w_lambda  wg_dist.mean  wg_dist.sd  worst_node.mean  \
query_index index                                                        
0           700    0.556480      1.734020    0.054781         0.910237   
            800    0.556480      1.734020    0.054781         0.910237   
            900    0.556480      1.734020    0.054781         0.910237   
            1000   0.556480      1.734020    0.054781         0.910237   
            1100   0.556480      1.734020    0.054781         0.910237   
...                     ...           ...         ...              ...   
99          999    0.199844      1.719397    0.086180         0.966667   
            1099   0.202941      1.709673    0.083441         0.966667   
            1199   0.478084      1.763438    0.071116         0.887500   
            1299   0.266574      1.715556    0.081032         0.979167   
            1399   0.506299      1.757148    0.064632         0.891667   

                   worst_node.mean.relative  worst_node.sd  \
query_index index                      